In [1]:
import altair as alt
import pandas as pd

# Define data URLs
school_data_url = "https://raw.githubusercontent.com/colmr/vis_class/master/primary_schools.csv"
ireland_map_url = "https://raw.githubusercontent.com/colmr/vis_class/master/Ireland_custom_topjson.json"

# Load the school data
school_data = alt.Data(url=school_data_url)

# Define the bar chart
county_click = alt.selection_interval(name="county_click")

bar_chart = alt.Chart(school_data).mark_bar().encode(
    y=alt.Y("County:N", axis=alt.Axis(title=None, labelFontSize=13)),
    x=alt.X(
        "count(Sex):Q",
        title="Number of Schools",
        axis=alt.Axis(titleFontSize=14, labelFontSize=13, titleFontWeight="bold"),
    ),
    color=alt.Color(
        "Sex:N",
        scale=alt.Scale(domain=["Boys", "Girls", "Mixed"], range=["#03fccf", "#bb4ec7", "#5157cf"]),
    ),
    order=alt.Order("Sex:N", sort="descending"),
    fillOpacity=alt.condition(county_click, alt.value(1), alt.value(0.3)),
).add_params(county_click).properties(
    width=100,
    height=505,
)

# Define the map chart
pts = alt.selection_interval(name="pts")

map_chart = (
    alt.Chart(alt.Data(url=ireland_map_url, format=alt.DataFormat(type="topojson", feature="IRL_adm1")))
    .mark_geoshape(stroke="white")
    .encode(color=alt.value("#b1b8ba"))
    .project("conicConformal")
    .properties(width=400, height=350)
)

points_chart = (
    alt.Chart(school_data)
    .mark_circle()
    .encode(
        longitude="Long:Q",
        latitude="Lat:Q",
        color=alt.Color(
            "Sex:N",
            scale=alt.Scale(domain=["Boys", "Girls", "Mixed"], range=["#03fccf", "#bb4ec7", "#5157cf"]),
        ),
        size=alt.Size(
            "Total:Q",
            scale=alt.Scale(type="threshold", domain=[251, 501, 750], range=[7, 20, 35, 50]),
        ),
    )
    .transform_filter(county_click)
    .transform_filter(pts)
    .add_params(pts)
)

layered_map_chart = alt.layer(map_chart, points_chart)

# Define the histogram chart
brush = alt.selection_interval(name="brush")

histogram_background = (
    alt.Chart(school_data)
    .mark_bar(color="lightgrey")
    .encode(
        x=alt.X("Total:Q", bin=alt.Bin(maxbins=50), title="Number of Students"),
        y=alt.Y("count():Q", title="Schools"),
    )
    .transform_filter(pts)
    .transform_filter(county_click)
)

histogram_foreground = (
    alt.Chart(school_data)
    .mark_bar(color="black")
    .encode(
        x=alt.X("Total:Q", bin=alt.Bin(maxbins=50), title="Number of Students"),
        y=alt.Y("count():Q", title="Schools"),
    )
    .transform_filter(pts)
    .transform_filter(county_click)
    .transform_filter(brush)
    .add_params(brush)
)

histogram_chart = alt.layer(histogram_background, histogram_foreground).properties(width=400, height=125)

# Combine map and histogram vertically
map_and_histogram = alt.vconcat(layered_map_chart, histogram_chart)

# Combine bar chart and map+histogram horizontally
final_chart = alt.hconcat(bar_chart, map_and_histogram).configure_legend(
    titleFontSize=15, labelFontSize=13
).properties(
    title=alt.TitleParams(
        text="Irish Primary School Explorer", anchor="middle", fontSize=20, fontWeight=800
    )
)

final_chart
final_chart.save("primary_schools.html")
